In [ ]:
# wget  -P $FLINK_HOME/lib https://repo1.maven.org/maven2/org/apache/flink/flink-parquet/1.15.3/flink-parquet-1.15.3.jar

# wget  -P $FLINK_HOME/lib https://repo1.maven.org/maven2/org/apache/flink/flink-sql-parquet/1.15.3/flink-sql-parquet-1.15.3.jar


In [1]:
import datetime
from pyflink.table import EnvironmentSettings, TableEnvironment, DataTypes, CsvTableSource

from pyflink.table.expressions import col
from pyflink.table.window import Over, GroupWindow
from pyflink.table.expressions import col, UNBOUNDED_RANGE, CURRENT_RANGE
from pyflink.table.udf import udf
# create a batch TableEnvironment
env_settings = EnvironmentSettings.in_batch_mode()
table_env = TableEnvironment.create(env_settings)

# this python/pyflink refers to python packages inside flink-dev, not refering to /opt/flink1-15.3/lib but we copied parquest jar there

# table_env.get_config().get_configuration().set_string("pipeline.jars", "file:///opt/flink-1.15.3/lib/flink-shaded-hadoop-3-uber-3.1.1.7.2.8.0-224-9.0.jar;file:///opt/flink-1.15.3/lib/flink-parquet-1.15.3.jar")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/flink-1.15.3/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.7.7/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [2]:
# InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
# 536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 8:26,2.55,17850,United Kingdom

source_ddl = """
            CREATE TABLE invoices (
                   InvoiceNo STRING,
                   StockCode  STRING,
                   Description  STRING,
                   Quantity DOUBLE,
                   InvoiceDate TIMESTAMP(3),
                   UnitPrice DOUBLE,
                   CustomerID STRING,
                   Country STRING,
                   WATERMARK FOR InvoiceDate AS InvoiceDate - INTERVAL '1' SECOND

                    ) WITH (
                      'connector' = 'filesystem',          
                     'path' = 'file:////home/training/flink-dev/data/ecommerce-clean.csv', 
                      'format' = 'csv'
                    )"""


table_env.execute_sql(source_ddl) 

invoices = table_env.from_path("invoices")
##############################
print('\nRegistered Tables List')
print(table_env.list_tables())

print('\nFinancial Trxs Schema')
invoices.print_schema()


Registered Tables List
['invoices']

Financial Trxs Schema
(
  `InvoiceNo` STRING,
  `StockCode` STRING,
  `Description` STRING,
  `Quantity` DOUBLE,
  `InvoiceDate` TIMESTAMP(3),
  `UnitPrice` DOUBLE,
  `CustomerID` STRING,
  `Country` STRING,
  WATERMARK FOR `InvoiceDate`: TIMESTAMP(3) AS `InvoiceDate` - INTERVAL '1' SECOND
)


In [3]:
# sink to write parquet files
# file:///home/rps/workshop/data/ecommerce-clean.parquet is a folder, will contain many partitions files
sink_ddl_parquet=f"""
    create table `EComParquetResult`(
                 InvoiceNo STRING,
                   StockCode  STRING,
                   Description  STRING,
                   Quantity DOUBLE,
                   InvoiceDate TIMESTAMP(3),
                   UnitPrice DOUBLE,
                   CustomerID STRING,
                   Country STRING
    ) with (
        'connector' = 'filesystem',
        'format' = 'parquet',
        'path' = 'file:////home/training/flink-dev/data/ecommerce-clean.parquet'
    )
    """

table_env.execute_sql(sink_ddl_parquet)

In [4]:
# emit the data from source to target

table_env.execute_sql("INSERT INTO EComParquetResult select * from invoices").wait()

invoices.execute_insert("EComParquetResult")

2023-01-18 16:51:52,145 INFO  org.apache.hadoop.io.compress.CodecPool                      [] - Got brand-new compressor [.snappy]
